# Product Recommendation
Reference: https://ieeexplore.ieee.org/document/5430993

In [1]:
import numexpr as ne
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm

In [2]:
tf.__version__

'2.3.1'

## Data Preprocessing

In [3]:
Y_data = pd.read_csv('data/Y.csv', header=None, names=['Rating','Movie','User'], dtype=np.int32) # training data
P_data = pd.read_csv('data/P.csv', header=None, names=['Rating','Movie','User'], dtype=np.int32) # test data ('probe-set' mentioned in paper)

In [4]:
display(Y_data.head())
display(P_data.head())
Y_data.shape, P_data.shape

,Rating,Movie,User
0,5,2,1
1,4,7,1
2,4,8,1
3,4,11,1
4,4,12,1


,Rating,Movie,User
0,3,6,1
1,5,96,1
2,3,1,2
3,3,33,3
4,5,42,4


((3399874, 3), (189699, 3))

In [5]:
print(Y_data['Rating'].unique().max(), Y_data['Movie'].unique().max(), Y_data['User'].unique().max())
print(P_data['Rating'].unique().max(), P_data['Movie'].unique().max(), P_data['User'].unique().max())

5 100 137328
5 100 137328


In [6]:
k, n = Y_data['Movie'].unique().max(), Y_data['User'].unique().max()
k, n

(100, 137328)

In [7]:
Z_sparse = tf.SparseTensor(indices=Y_data[['Movie', 'User']].values-1, values=Y_data['Rating'].values, dense_shape=[k, n])
Z_sparse = tf.cast(Z_sparse, tf.float64)

In [8]:
# use dense matrices for faster linear transformations since all matrices can fit in memory
Z = tf.sparse.to_dense(Z_sparse, validate_indices=False)
Z

<tf.Tensor: shape=(100, 137328), dtype=float64, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [5., 0., 0., ..., 0., 0., 3.],
       [0., 0., 0., ..., 3., 0., 0.],
       ...,
       [5., 0., 0., ..., 4., 0., 4.],
       [4., 0., 3., ..., 0., 0., 4.],
       [3., 4., 0., ..., 4., 5., 4.]])>

In [9]:
Y_data_user_ids = Y_data['User'].values
P_data_user_ids = P_data['User'].values

data_preprocessed = list()
for t in tqdm(range(n)):
    movie_ids_indices = Y_data[ne.evaluate(f'Y_data_user_ids == {t+1}')]['Movie'].values - 1
    H_yt = tf.constant(np.identity(k)[movie_ids_indices], dtype=tf.float64)
    H_xt = tf.constant(np.delete(np.identity(k), movie_ids_indices, 0), dtype=tf.float64)
    
    k_t = tf.constant(H_yt.shape[0], dtype=tf.float64)
    Z_t = tf.expand_dims(Z[:, t], axis=1) 
    y_t = tf.matmul(H_yt, Z_t)
    x_t = tf.matmul(H_xt, Z_t)
    
    movie_ids_t = P_data[ne.evaluate(f'P_data_user_ids == {t+1}')]['Movie'].values
    labels_t = tf.expand_dims(P_data[ne.evaluate(f'P_data_user_ids == {t+1}')]['Rating'].values, axis=1)
    data_preprocessed.append((H_yt, H_xt, tf.transpose(H_yt), tf.transpose(H_xt), k_t, Z_t, y_t, x_t, movie_ids_t, labels_t))
        
del Y_data
del P_data
del Z_sparse
del Y_data_user_ids
del P_data_user_ids

100%|██████████| 137328/137328 [10:28<00:00, 218.59it/s]


## Initialization
$\mu$ has 1 type available <br />
R has 4 types available

In [10]:
# initial estimate of mu
N = 0
H_yty_t = 0
    
for (H_yt, H_xt, H_yt_trans, H_xt_trans, k_t, Z_t, y_t, x_t, movie_ids_t, labels_t) in tqdm(data_preprocessed):
    N += tf.matmul(H_yt_trans, H_yt)
    H_yty_t += tf.matmul(H_yt_trans, y_t)

100%|██████████| 137328/137328 [00:14<00:00, 9359.09it/s] 


In [11]:
# The ith diagonal element of N equals the total number of ratings of the ith product.
N

<tf.Tensor: shape=(100, 100), dtype=float64, numpy=
array([[20017.,     0.,     0., ...,     0.,     0.,     0.],
       [    0., 23917.,     0., ...,     0.,     0.,     0.],
       [    0.,     0., 31634., ...,     0.,     0.,     0.],
       ...,
       [    0.,     0.,     0., ..., 60896.,     0.,     0.],
       [    0.,     0.,     0., ...,     0., 61521.,     0.],
       [    0.,     0.,     0., ...,     0.,     0., 64506.]])>

In [12]:
mu_hat0 = tf.matmul(tf.linalg.inv(N), H_yty_t)
tf.transpose(mu_hat0)

<tf.Tensor: shape=(1, 100), dtype=float64, numpy=
array([[3.45266523, 3.57674457, 3.28788645, 3.90478757, 3.79035475,
        3.44415598, 3.19071562, 4.52835008, 3.82013753, 3.6159503 ,
        3.40382731, 3.83725101, 4.07603884, 4.22836664, 3.35395465,
        4.0645276 , 3.72119599, 3.48700861, 4.16388921, 3.40982441,
        3.86926003, 3.43583485, 3.20324443, 4.08487897, 3.23199846,
        3.88664794, 4.33189497, 4.38358165, 4.31638739, 3.86591733,
        4.33975717, 3.89147883, 3.70029269, 3.36247781, 4.32901523,
        4.06706884, 4.56922029, 3.77104091, 3.68586682, 3.84532386,
        4.3454114 , 3.90999207, 3.39949928, 3.60786807, 3.96267104,
        4.14386102, 3.4072049 , 3.7040225 , 4.00350359, 4.64280228,
        3.21623279, 3.77238583, 4.26565116, 4.45377313, 3.83848945,
        3.79374176, 3.7629172 , 3.88698608, 3.80041727, 4.34696995,
        3.80469565, 3.84624795, 3.64122601, 3.27221683, 3.42333499,
        3.71631568, 3.20698918, 4.45410441, 4.26541296, 3.86109184

In [13]:
# initial estimates of R (4 types available)
R_hat0_1 = tf.eye(k, dtype=tf.float64)
R_hat0_1

<tf.Tensor: shape=(100, 100), dtype=float64, numpy=
array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])>

In [14]:
S = 0
for (H_yt, H_xt, H_yt_trans, H_xt_trans, k_t, Z_t, y_t, x_t, movie_ids_t, labels_t) in tqdm(data_preprocessed):
    Hytmu_hat0 = tf.matmul(H_yt, mu_hat0)
    S += H_yt_trans @ (y_t - Hytmu_hat0) @ tf.transpose(y_t - Hytmu_hat0) @ H_yt

100%|██████████| 137328/137328 [00:21<00:00, 6323.46it/s]


In [15]:
# diag_S is the diagonal matrix consisting of the diagonal elements of S
diag_S = tf.linalg.diag(tf.linalg.tensor_diag_part(S))
R_hat0_2 = tf.matmul(tf.linalg.inv(N), diag_S)
R_hat0_2

<tf.Tensor: shape=(100, 100), dtype=float64, numpy=
array([[1.72440427, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.94219113, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.43659411, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.18291506, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.03485685,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.26227449]])>

In [16]:
# R_hat0_3 is not a good initializer when rating variances are far from one
R_hat0_3 = tf.matmul(tf.linalg.sqrtm(tf.linalg.inv(diag_S)), tf.matmul(S, tf.linalg.sqrtm(tf.linalg.inv(diag_S))))
R_hat0_3

<tf.Tensor: shape=(100, 100), dtype=float64, numpy=
array([[ 1.        ,  0.07418256, -0.01158277, ..., -0.01462987,
        -0.02215371, -0.01844816],
       [ 0.07418256,  1.        ,  0.03674347, ...,  0.0256191 ,
         0.03563234,  0.03926307],
       [-0.01158277,  0.03674347,  1.        , ...,  0.10955311,
         0.12823359,  0.15560634],
       ...,
       [-0.01462987,  0.0256191 ,  0.10955311, ...,  1.        ,
         0.19781317,  0.15164928],
       [-0.02215371,  0.03563234,  0.12823359, ...,  0.19781317,
         1.        ,  0.18995689],
       [-0.01844816,  0.03926307,  0.15560634, ...,  0.15164928,
         0.18995689,  1.        ]])>

In [17]:
R_hat0_4 = tf.matmul(tf.linalg.sqrtm(tf.linalg.inv(N)), tf.matmul(S, tf.linalg.sqrtm(tf.linalg.inv(N))))
R_hat0_4

<tf.Tensor: shape=(100, 100), dtype=float64, numpy=
array([[ 1.72440427,  0.09455639, -0.01823052, ..., -0.02089473,
        -0.02959417, -0.02721758],
       [ 0.09455639,  0.94219113,  0.04274809, ...,  0.02704644,
         0.03518471,  0.04281842],
       [-0.01823052,  0.04274809,  1.43659411, ...,  0.14281326,
         0.15635399,  0.20954206],
       ...,
       [-0.02089473,  0.02704644,  0.14281326, ...,  1.18291506,
         0.21886288,  0.18530794],
       [-0.02959417,  0.03518471,  0.15635399, ...,  0.21886288,
         1.03485685,  0.21710614],
       [-0.02721758,  0.04281842,  0.20954206, ...,  0.18530794,
         0.21710614,  1.26227449]])>

## Expectation Maximization Algorithm

In [18]:
LOG_2PI = tf.math.log(2*tf.constant(np.pi, dtype=tf.float64))

@tf.function(experimental_relax_shapes=True)
def run_graph_em(mu, R, y_t, H_xt, H_xt_trans, H_yt, H_yt_trans, k_t):
    # for R estimation
    R_xt = H_xt @ R @ H_xt_trans
    R_yt = H_yt @ R @ H_yt_trans
    R_yt_det = tf.linalg.det(R_yt)
    R_yt_inv = tf.linalg.inv(R_yt)
    R_xtyt = H_xt @ R @ H_yt_trans

    mu_yt = tf.matmul(H_yt, mu)
    mu_xt = tf.matmul(H_xt, mu)

    X_t_hat = R_xtyt @ R_yt_inv @ (y_t - mu_yt) + mu_xt
    Z_t_hat = H_yt_trans @ y_t + H_xt_trans @ X_t_hat
    
    R_hat_sum_part = (Z_t_hat - mu) @ tf.transpose(Z_t_hat - mu) \
                        + H_xt_trans @ (R_xt - R_xtyt @ R_yt_inv @ tf.transpose(R_xtyt)) @ H_xt

    # for mu estimation
    Hyt_trans_Ryt_inv_Hyt_sum_part = H_yt_trans @ R_yt_inv @ H_yt
    Hyt_trans_Ryt_inv_yt_sum_part = H_yt_trans @ R_yt_inv @ y_t
    
    # for log likelihood calculation
    log_p_hat_part = -1/2*(tf.math.log(R_yt_det) + tf.transpose(y_t - mu_yt) @ R_yt_inv @ (y_t - mu_yt) + k_t*LOG_2PI) 
    
    return R_hat_sum_part, Hyt_trans_Ryt_inv_Hyt_sum_part, Hyt_trans_Ryt_inv_yt_sum_part, log_p_hat_part

In [19]:
def expectation_maximization(mu, R):
    Hyt_trans_Ryt_inv_Hyt_sum = 0
    Hyt_trans_Ryt_inv_yt_sum = 0
    R_hat_sum = 0
    log_p_hat = 0
    
    for (H_yt, H_xt, H_yt_trans, H_xt_trans, k_t, Z_t, y_t, x_t, movie_ids_t, labels_t) in tqdm(data_preprocessed):
        R_hat_sum_part, Hyt_trans_Ryt_inv_Hyt_sum_part, Hyt_trans_Ryt_inv_yt_sum_part, log_p_hat_part = \
            run_graph_em(mu, R, y_t, H_xt, H_xt_trans, H_yt, H_yt_trans, k_t)
        
        R_hat_sum += R_hat_sum_part
        Hyt_trans_Ryt_inv_Hyt_sum += Hyt_trans_Ryt_inv_Hyt_sum_part
        Hyt_trans_Ryt_inv_yt_sum += Hyt_trans_Ryt_inv_yt_sum_part
        log_p_hat += log_p_hat_part
        
    R_hat = R_hat_sum / n
    mu_hat = tf.matmul(tf.linalg.inv(Hyt_trans_Ryt_inv_Hyt_sum), Hyt_trans_Ryt_inv_yt_sum)    
    return mu_hat, R_hat, log_p_hat

In [20]:
delta = 0.0005
mu = mu_hat0
R = R_hat0_4
log_p = tf.constant(-np.inf, dtype=tf.float64)

for i in range(30):
    if i % 5 == 0:
        print(f'iteration: {i}')
    
    mu_hat, R_hat, log_p_hat = expectation_maximization(mu, R)
    convergence_criterion = log_p_hat/n - log_p/n < delta
    
    print('normalized log_p_hat:', (log_p_hat/n).numpy().flatten()[0])
    print('normalized log_p:    ', (log_p/n).numpy().flatten()[0])
    print('convergence gap:     ', (log_p_hat/n - log_p/n).numpy().flatten()[0])
    
    if convergence_criterion:
        break
        
    # use new estimattions for next iteration
    mu = mu_hat
    R = R_hat
    log_p = log_p_hat

  0%|          | 1/137328 [00:00<7:22:28,  5.17it/s]

iteration: 0


  0%|          | 107/137328 [00:00<02:09, 1061.38it/s]

normalized log_p_hat: -32.23267266620053
normalized log_p:     -inf
convergence gap:      inf


  0%|          | 104/137328 [00:00<02:12, 1038.99it/s]

...

normalized log_p_hat: -31.415080208803396
normalized log_p:     -31.4157938181978
convergence gap:      0.0007136093944026811


  0%|          | 128/137328 [00:00<01:47, 1278.64it/s]

normalized log_p_hat: -31.414490994453352
normalized log_p:     -31.415080208803396
convergence gap:      0.0005892143500432212
iteration: 25


100%|██████████| 137328/137328 [01:51<00:00, 1230.33it/s]

normalized log_p_hat: -31.414002583121093
normalized log_p:     -31.414490994453352
convergence gap:      0.0004884113322596306


In [21]:
# 26 iterations, ~38 min
np.save('results/em_mu.npy', mu_hat)
np.save('results/em_R.npy', R_hat)
np.save('results/em_log_p.npy', log_p_hat)

## McMichael’s Algorithm

In [22]:
@tf.function(experimental_relax_shapes=True)
def run_graph_mcmichael(mu, R, y_t, H_yt, H_yt_trans, k_t):
    # for R estimation
    R_yt = H_yt @ R @ H_yt_trans
    R_yt_det = tf.linalg.det(R_yt)
    R_yt_inv = tf.linalg.inv(R_yt)
    mu_yt = tf.matmul(H_yt, mu)
    log_p_gradient_part = H_yt_trans @ (R_yt_inv - R_yt_inv @ (y_t - mu_yt) @ tf.transpose(y_t - mu_yt) @ R_yt_inv) @ H_yt

    # for mu estimation
    Hyt_trans_Ryt_inv_Hyt_sum_part = H_yt_trans @ R_yt_inv @ H_yt
    Hyt_trans_Ryt_inv_yt_sum_part = H_yt_trans @ R_yt_inv @ y_t
    
    # for log likelihood calculation
    log_p_hat_part = -1/2*(tf.math.log(R_yt_det) + tf.transpose(y_t - mu_yt) @ R_yt_inv @ (y_t - mu_yt) + k_t*LOG_2PI)
    
    return log_p_gradient_part, Hyt_trans_Ryt_inv_Hyt_sum_part, Hyt_trans_Ryt_inv_yt_sum_part, log_p_hat_part

In [23]:
def mcmichael(mu, R):
    gamma = 0.00001
    Hyt_trans_Ryt_inv_Hyt_sum = 0
    Hyt_trans_Ryt_inv_yt_sum = 0
    log_p_gradient = 0
    log_p_hat = 0

    for (H_yt, H_xt, H_yt_trans, H_xt_trans, k_t, Z_t, y_t, x_t, movie_ids_t, labels_t) in tqdm(data_preprocessed):
        log_p_gradient_part, Hyt_trans_Ryt_inv_Hyt_sum_part, Hyt_trans_Ryt_inv_yt_sum_part, log_p_hat_part = \
            run_graph_mcmichael(mu, R, y_t, H_yt, H_yt_trans, k_t)
        
        log_p_gradient += log_p_gradient_part
        Hyt_trans_Ryt_inv_Hyt_sum += Hyt_trans_Ryt_inv_Hyt_sum_part
        Hyt_trans_Ryt_inv_yt_sum += Hyt_trans_Ryt_inv_yt_sum_part
        log_p_hat += log_p_hat_part
        
    R_hat = R + gamma*(R @ (-1/2*log_p_gradient) @ R)
    mu_hat = tf.matmul(tf.linalg.inv(Hyt_trans_Ryt_inv_Hyt_sum), Hyt_trans_Ryt_inv_yt_sum)
    return mu_hat, R_hat, log_p_hat

In [24]:
delta = 0.0005
mu = mu_hat0
R = R_hat0_4
log_p = tf.constant(-np.inf, dtype=tf.float64)

for i in range(40):
    if i % 5 == 0:
        print(f'iteration: {i}')
    
    mu_hat, R_hat, log_p_hat = mcmichael(mu, R)
    convergence_criterion = log_p_hat/n - log_p/n < delta
    
    print('normalized log_p_hat:', (log_p_hat/n).numpy().flatten()[0])
    print('normalized log_p:    ', (log_p/n).numpy().flatten()[0])
    print('convergence gap:     ', (log_p_hat/n - log_p/n).numpy().flatten()[0])
    
    if convergence_criterion:
        break
        
    # use new estimattions for next iteration
    mu = mu_hat
    R = R_hat
    log_p = log_p_hat

  0%|          | 0/137328 [00:00<?, ?it/s]

iteration: 0


  0%|          | 168/137328 [00:00<01:21, 1673.87it/s]

normalized log_p_hat: -32.23267266620053
normalized log_p:     -inf
convergence gap:      inf


  0%|          | 165/137328 [00:00<01:23, 1647.38it/s]

...

normalized log_p_hat: -31.416111707402408
normalized log_p:     -31.41672871330004
convergence gap:      0.0006170058976309178


  0%|          | 182/137328 [00:00<01:15, 1809.47it/s]

normalized log_p_hat: -31.415571985818687
normalized log_p:     -31.416111707402408
convergence gap:      0.0005397215837206204


100%|██████████| 137328/137328 [01:15<00:00, 1807.30it/s]

normalized log_p_hat: -31.415099001376582
normalized log_p:     -31.415571985818687
convergence gap:      0.00047298444210497337


In [25]:
# 35 iterations, ~38 min
np.save('results/mcmichael_mu.npy', mu_hat)
np.save('results/mcmichael_R.npy', R_hat)
np.save('results/mcmichael_log_p.npy', log_p_hat)

## Evaluation

In [26]:
@tf.function(experimental_relax_shapes=True)
def run_graph_square_error(mu, R, movie_ids_t, labels_t, y_t, H_xt, H_xt_trans, H_yt, H_yt_trans):
    # calculate X_t_hat
    R_xt = H_xt @ R @ H_xt_trans
    R_yt = H_yt @ R @ H_yt_trans
    R_yt_inv = tf.linalg.inv(R_yt)
    R_xtyt = H_xt @ R @ H_yt_trans

    mu_yt = tf.matmul(H_yt, mu)
    mu_xt = tf.matmul(H_xt, mu)

    X_t_hat = R_xtyt @ R_yt_inv @ (y_t - mu_yt) + mu_xt
    
    # clip ratings
    predictions_t = tf.gather(tf.matmul(H_xt_trans, X_t_hat), indices=movie_ids_t-1)
    predictions_t = tf.clip_by_value(predictions_t, 1, 5)
    
    return tf.matmul(tf.transpose(labels_t - predictions_t), labels_t - predictions_t)

In [27]:
def evaluate(mu, R):
    square_error = 0
    l = 0
    for (H_yt, H_xt, H_yt_trans, H_xt_trans, k_t, Z_t, y_t, x_t, movie_ids_t, labels_t) in tqdm(data_preprocessed):
        square_error += run_graph_square_error(mu, R, movie_ids_t, tf.cast(labels_t, dtype=tf.float64), y_t, H_xt, H_xt_trans, H_yt, H_yt_trans)
        l += len(labels_t)
    return np.sqrt(square_error/l)

In [28]:
em_mu = np.load('results/em_mu.npy')
em_R = np.load('results/em_R.npy')
rmse = evaluate(em_mu, em_R)
rmse

100%|██████████| 137328/137328 [01:34<00:00, 1454.87it/s]


array([[0.91700701]])

In [29]:
mcmichael_mu = np.load('results/mcmichael_mu.npy')
mcmichael_R = np.load('results/mcmichael_R.npy')
rmse = evaluate(mcmichael_mu, mcmichael_R)
rmse

100%|██████████| 137328/137328 [01:36<00:00, 1425.03it/s]


array([[0.91701472]])